<a href="https://colab.research.google.com/github/blackgreenc/java/blob/master/KerasModel_Java_pj_ipynd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)
# MNIST Datasets 불러오기 
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

2.7.0
11501568/11490434 [==============================] - 0s 0us/step


In [2]:
# input data normalization
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
# model create
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# model compile
mlp_model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [4]:
# model
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
# model training
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2567 - accuracy: 0.9266
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1141 - accuracy: 0.9664
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0796 - accuracy: 0.9756
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0593 - accuracy: 0.9821
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0450 - accuracy: 0.9861


In [6]:
# model evaluation
mlp_model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0798 - accuracy: 0.9755 - 490ms/epoch - 2ms/step


[0.07976802438497543, 0.9754999876022339]

In [7]:
# keras model convert to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model) # create converter 
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp83fnxq8b/assets


In [8]:
# keras model save
with open('./keras_model.tflite', 'wb') as f:
    f.write(tflite_model) 

In [9]:
#tf.saved_model.save(mlp_model, "./mlp_model/")
mlp_model.save("./mlp_model/")

INFO:tensorflow:Assets written to: ./mlp_model/assets


INFO:tensorflow:Assets written to: ./mlp_model/assets


In [10]:
saved_model = tf.keras.models.load_model("./mlp_model/")

In [11]:
# keras model convert to tflite model
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/") # create converter 
tflite_model = converter.convert()

In [12]:
# SavedModel save
with open('./saved_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [20]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs + 1
inc = Inc()

In [21]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1
inc_g = Inc_Graph()

In [24]:
inc_g2 = tf.function(inc)

In [36]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype = tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [40]:
concrete_fun = inc_g2.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype = tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

IndexError: ignored

In [41]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

graph_model = tf.function(mlp_model)
concrete_func = graph_model.get_concrete_function(tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype = mlp_model.inputs[0].dtype))

In [42]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation = 'softmax')
    
    @tf.function
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)

mlp_model = MLP_Model()

In [43]:
concrete_func = mlp_model.call.get_concrete_function(
    tf.TensorSpec(shape = mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))

TypeError: ignored

In [16]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('./concrete_func_model.tflite', 'wb') as f:
    f.write(tflite_model)